In [ ]:
class ToolDefinition:
    """Helper class for defining tools with Pydantic models."""
    
    @staticmethod
    def from_function(
        function: Callable,
        params_model: Optional[Type[BaseModel]] = None,
        response_model: Optional[Type[BaseModel]] = None,
        name: Optional[str] = None,
        description: Optional[str] = None
    ) -> Tuple[Callable, Dict[str, Any]]:
        """
        Create a tool definition from a function and optional Pydantic models.
        
        Args:
            function: The function implementation
            params_model: Optional Pydantic model for parameters
            response_model: Optional Pydantic model for response
            name: Optional custom name
            description: Optional custom description
            
        Returns:
            Tuple of (function, schema)
        """
        if params_model:
            # Use Pydantic model for schema
            import inspect
            
            # Get function metadata
            func_name = name or function.__name__
            func_description = description or inspect.getdoc(function) or ""
            
            # Convert Pydantic models to Gemini schemas
            params_schema = GeminiUtils.pydantic_to_schema(params_model)
            
            # Create function definition
            schema = {
                "name": func_name,
                "description": func_description,
                "parameters": params_schema
            }
            
            # Add response schema if provided
            if response_model:
                response_schema = GeminiUtils.pydantic_to_schema(response_model)
                schema["response"] = response_schema
                
            return (function, schema)
        else:
            # Let GeminiUtils extract schema from function signature
            schema = GeminiUtils.python_to_function(function, name, description)
            return (function, schema)
        
        # Process tools to standardized format
        processed_tools = []
        tool_callables = {}
        
        if tools:
            for tool in tools:
                if isinstance(tool, tuple) and len(tool) >= 2 and callable(tool[0]):
                    # Case: (function, params_model, [response_model]) tuple
                    function = tool[0]
                    params_model = tool[1]
                    response_model = tool[2] if len(tool) > 2 else None
                    
                    func, schema = ToolDefinition.from_function(
                        function=function,
                        params_model=params_model,
                        response_model=response_model
                    )
                    
                    processed_tools.append(schema)
                    tool_callables[schema["name"]] = func
                    
                elif callable(tool):
                    # Case: Plain function
                    schema = GeminiUtils.python_to_function(tool)
                    processed_tools.append(schema)
                    tool_callables[schema["name"]] = tool
                    
                elif isinstance(tool, dict) and "name" in tool:
                    # Case: Pre-made schema
                    processed_tools.append(tool)
                    # Note: Function implementation must be provided separately
                    
                else:
                    logger.warning(f"Ignoring unsupported tool type: {type(tool)}")